# Analiza podatkov s knjižnico Pandas

Spodaj je pregled najosnovnejših metod, ki jih ponuja knjižnica Pandas. Vsaka od naštetih metod ponuja še cel kup dodatnih možnosti, ki so natančno opisane v [uradni dokumentaciji](http://pandas.pydata.org/pandas-docs/stable/). Z branjem dokumentacije se vam seveda najbolj splača začeti pri [uvodih](http://pandas.pydata.org/pandas-docs/stable/tutorials.html).

## Predpriprava

In [ ]:
# naložimo paket
import pandas as pd

# naložimo razpredelnico, s katero bomo delali
filmi = pd.read_csv('obdelani-podatki/filmi.csv', index_col='id')

# ker bomo delali z velikimi razpredelnicami, povemo, da naj se vedno izpiše le 20 vrstic
pd.options.display.max_rows = 20

## Osnovni izbori elementov razpredelnic

Z metodo `.head(n=5)` pogledamo prvih `n`, z metodo `.tail(n=5)` pa zadnjih `n` vrstic razpredelnice.

In [ ]:
filmi.head(10)

In [ ]:
filmi.tail()

Z rezinami pa dostopamo do izbranih vrstic.

In [ ]:
filmi[3:10:2]

Z indeksiranjem razpredelnice dostopamo do posameznih stolpcev.

In [ ]:
filmi['ocena']

Do stolpcev pogosto dostopamo, zato lahko uporabimo tudi krajši zapis.

In [ ]:
filmi.ocena

Če želimo več stolpcev, moramo za indeks podati seznam vseh oznak.

In [ ]:
filmi[['naslov', 'ocena']]

Do vrednosti z indeksom `i` dostopamo z `.iloc[i]`, do tiste s ključem `k` pa z `.loc[k]`.

In [ ]:
filmi.iloc[120]

In [ ]:
filmi.loc[97576]

## Filtriranje

Izbor določenih vrstic razpredelnice naredimo tako, da za indeks podamo stolpec logičnih vrednosti, ki ga dobimo z običajnimi operacijami. V vrnjeni razpredelnici bodo ostale vrstice, pri katerih je v stolpcu vrednost `True`.

In [ ]:
filmi.ocena >= 8

In [ ]:
filmi[filmi.ocena >= 8]

In [ ]:
filmi[(filmi.leto > 2010) & (filmi.ocena > 8)]

### Naloga

Poiščite filme, ki si jih želimo izogniti za vsako ceno, torej tiste, ki so daljši od dveh ur in imajo oceno pod 6.5.

In [ ]:
filmi[(filmi.ocena < 6) & (filmi.dolzina > 150)]

## Urejanje

Razpredelnico urejamo z metodo `.sort_values`, ki ji podamo ime ali seznam imen stolpcev, po katerih želimo urejati. Po želji lahko tudi povemo, kateri stolpci naj bodo urejeni naraščajoče in kateri padajoče.

In [ ]:
filmi.sort_values('dolzina')

In [ ]:
# najprej uredi padajoče po oceni, pri vsaki oceni pa še naraščajoče po letu
filmi.sort_values(['ocena', 'leto'], ascending=[False, True])

## Združevanje

Z metodo `.groupby` ustvarimo razpredelnico posebne vrste, v katerem so vrstice združene glede na skupno lastnost.

In [ ]:
filmi_po_letih = filmi.groupby('leto')

In [ ]:
filmi_po_letih

In [ ]:
# povprečna ocena vsakega leta
filmi_po_letih.ocena.mean()

Če želimo, lahko združujemo tudi po izračunanih lastnostih. Izračunajmo stolpec in ga shranimo v razpredelnico.

In [ ]:
filmi['desetletje'] = 10 * (filmi.leto // 10)

In [ ]:
filmi

In [ ]:
filmi_po_desetletjih = filmi.groupby('desetletje')

Preštejemo, koliko filmov je bilo v vsakem desetletju. Pri večini stolpcev dobimo iste številke, ker imamo v vsakem stolpcu enako vnosov. Če kje kakšen podatek manjkal, je številka manjša.

In [ ]:
filmi_po_desetletjih.count()

Če želimo dobiti le število članov posamezne skupine, uporabimo metodo `.size()`. V tem primeru dobimo le stolpec, ne razpredelnice.

In [ ]:
filmi_po_desetletjih.size()

Pogledamo povprečja vsakega desetletja. Dobimo povprečno leto, povprečno dolžino in oceno. Povprečnega naslova ne dobimo, ker se ga ne da izračunati, zato ustreznega stolpca ni.

In [ ]:
filmi_po_desetletjih.mean()

### Naloga

Izračunajte število filmov posamezne dolžine, zaokrožene na 5 minut.

## Risanje grafov

In [ ]:
# vključimo risanje grafov (če stvari začnejo delati počasneje, izklopimo možnost inline)
%matplotlib inline

Običajen graf dobimo z metodo `plot`. Uporabljamo ga, kadar želimo prikazati spreminjanje vrednosti v odvisnosti od zvezne spremenljivke. Naša hipoteza je, da so zlata leta filma mimo. Graf to potrjuje.

In [ ]:
filmi[filmi.ocena > 8.5].groupby('desetletje').size().plot()

Razsevni diagram dobimo z metodo `plot.scatter`. Uporabljamo ga, če želimo ugotoviti povezavo med dvema spremenljivkama.

In [ ]:
filmi.plot.scatter('dolzina', 'ocena')

In [ ]:
filmi[filmi.dolzina < 250].plot.scatter('dolzina', 'ocena')

Stolpčni diagram dobimo z metodo `plot.bar`. Uporabljamo ga, če želimo primerjati vrednosti pri diskretnih (običajno kategoričnih) spremenljivkah. Pogosto je koristno, da graf uredimo po vrednostih.

In [ ]:
filmi.sort_values('zasluzek', ascending=False).head(20).plot.bar(x='naslov', y='zasluzek')

### Naloga

Narišite grafe, ki ustrezno kažejo:
- Povezavo med IMDB in metascore oceno
- Spreminjanje povprečne dolžine filmov skozi leta

## Stikanje

In [ ]:
osebe = pd.read_csv('obdelani-podatki/osebe.csv', index_col='id')
vloge = pd.read_csv('obdelani-podatki/vloge.csv')
zanri = pd.read_csv('obdelani-podatki/zanri.csv')

Razpredelnice stikamo s funkcijo `merge`, ki vrne razpredelnico vnosov iz obeh tabel, pri katerih se vsi istoimenski podatki ujemajo.

In [ ]:
vloge[vloge.film == 12349]

In [ ]:
zanri[zanri.film == 12349]

In [ ]:
pd.merge(vloge, zanri).head(20)

V osnovi vsebuje staknjena razpredelnica le tiste vnose, ki se pojavijo v obeh tabelah. Temu principu pravimo notranji stik (_inner join_). Lahko pa se odločimo, da izberemo tudi tiste vnose, ki imajo podatke le v levi tabeli (_left join_), le v desni tabeli (_right join_) ali v vsaj eni tabeli (_outer join_). Če v eni tabeli ni vnosov, bodo v staknjeni tabeli označene manjkajoče vrednosti. Ker smo v našem primeru podatke jemali iz IMDBja, kjer so za vsak film določeni tako žanri kot vloge, do razlik ne pride.

Včasih želimo stikati tudi po stolpcih z različnimi imeni. V tem primeru funkciji `merge` podamo argumenta `left_on` in `right_on`.

In [ ]:
pd.merge(pd.merge(vloge, zanri), osebe, left_on='oseba', right_on='id')

Poglejmo, katera osebe so nastopale v največ komedijah.

In [ ]:
zanri_oseb = pd.merge(pd.merge(vloge, zanri), osebe, left_on='oseba', right_on='id')
zanri_oseb[
    (zanri_oseb.zanr == 'Comedy') &
    (zanri_oseb.vloga == 'igralec')
].groupby(
    'ime'
).size(
).sort_values(
    ascending=False
).head(20)

### Naloga

- Izračunajte povprečno oceno vsakega žanra.
- Kateri režiserji snemajo najdonosnejše filme?

## Zahtevnejši primer: naivni Bayesov klasifikator

Zanima nas, ali lahko iz opisa filma napovemo njegove žanre. Gre za _klasifikacijski problem_, saj želimo filme klasificirati v žanre, naša naloga pa je napisati ustrezen program, ki mu pravimo _klasifikator_. Da zadevo naredimo bolj obvladljivo, bomo opis predstavili le z množico korenov besed, ki se v opisu pojavljajo.

In [ ]:
def koren_besede(beseda):
    beseda = ''.join(znak for znak in beseda if znak.isalpha())
    if not beseda:
        return '$'
    konec = len(beseda) - 1
    if beseda[konec] in 'ds':
        konec -= 1
    while konec >= 0 and beseda[konec] in 'aeiou':
        konec -= 1
    return beseda[:konec + 1]

def koreni_besed(niz):
    return {
        koren_besede(beseda) for beseda in niz.replace('-', ' ').lower().split() if beseda
    }

In [ ]:
koreni_besed("In 1938, after his father Professor Henry Jones, Sr. goes missing while pursuing the Holy Grail, Indiana Jones finds himself up against Adolf Hitler's Nazis again to stop them obtaining its powers.")

Zanimala nas bo torej verjetnost, da ima film žanr $Ž_i$ ob pogoju, da njegov opis vsebuje korene $K_1, \ldots, K_m$, torej

$$P(Ž_i | K_1 \cap \cdots \cap K_n)$$

Pri tem se bomo poslužili Bayesovega izreka

$$P(A | B) = \frac{P(A \cap B)}{P(B)} = \frac{P(B | A) \cdot P(A)}{P(B)}$$

zaradi česar našemu klasifikatorju pravimo _Bayesov klasifikator_. Velja

$$P(Ž_i | K_1 \cap \cdots \cap K_n) = \frac{P(K_1 \cap \cdots \cap K_n | Ž_i) \cdot P(Ž_i)}{P(K_1 \cap \cdots \cap K_n)}$$

Nadalje si nalogo poenostavimo s predpostavko, da so pojavitve besed med seboj neodvisne. To sicer ni res, na primer ob besedi _treasure_ se bolj pogosto pojavlja beseda _hidden_ kot na primer _boring_, zato pravimo, da je klasifikator _naiven_. Ob tej predpostavki velja:

$$P(K_1 \cap \cdots \cap K_n | Ž_i) = P(K_1 | Ž_i) \cdot \cdots \cdot P(K_n | Ž_i)$$

oziroma

$$P(Ž_i | K_1 \cap \cdots \cap K_n) = \frac{P(K_1 | Ž_i) \cdot \cdots \cdot P(K_n | Ž_i) \cdot P(Ž_i)}{P(K_1 \cap \cdots \cap K_n)}$$

Filmu, katerega opis vsebuje korene $K_1, \dots, K_n$ bomo priredili tiste žanre $Ž_i$, pri katerih je dana verjetnost največja. Ker imenovalec ni odvisen od žanra, moramo torej za vsak $Ž_i$ izračunati le števec:

$$P(K_1 | Ž_i) \cdot \cdots \cdot P(K_n | Ž_i) \cdot P(Ž_i)$$

Vse te podatke znamo izračunati, zato se lahko lotimo dela.

Verjetnost posameznega žanra $P(Ž)$ izračunamo brez večjih težav:

In [ ]:
verjetnosti_zanrov = zanri.groupby('zanr').size() / len(filmi)
verjetnosti_zanrov

Verjetnosti $P(K|Ž)$ bomo shranili v razpredelnico, v kateri bodo vrstice ustrezale korenom $K$, stolpci pa žanrom $Ž$. Najprej moramo poiskati vse filme, ki imajo žanr $Ž$, njihov opis pa vsebuje koren $K$. Vzemimo vse opise filmov:

In [ ]:
filmi.opis

To vrsto nizov pretvorimo v vrsto množic besed. Uporabimo metodo `apply`, ki dano funkcijo uporabi na vsakem vnosu.

In [ ]:
filmi.opis.apply(
    koreni_besed
)

Po nekaj [iskanja po internetu](https://stackoverflow.com/questions/30885005/pandas-series-of-lists-to-one-series) in masiranja pridemo do iskane razpredelnice:

In [ ]:
koreni_filmov = filmi.opis.apply(
    koreni_besed
).apply(
    list
).apply(
    pd.Series
).stack(
).reset_index(
    level='id'
).rename(columns={
    'id': 'film',
    0: 'koren',
})
koreni_filmov

Razpredelnico združimo z razpredelnico žanrov, da dobimo razpredelnico korenov žanrov.

In [ ]:
koreni_zanrov = pd.merge(
    koreni_filmov,
    zanri
)[['koren', 'zanr']]
koreni_zanrov

S pomočjo funkcije `crosstab` preštejemo, kolikokrat se vsaka kombinacija pojavi.

In [ ]:
pojavitve_korenov_po_zanrih = pd.crosstab(koreni_zanrov.koren, koreni_zanrov.zanr)
pojavitve_korenov_po_zanrih

Iskane verjetnosti sedaj dobimo tako, da vsak stolpec delimo s številom filmov danega žanra. Da ne bomo dobili ničelne verjetnosti pri korenih, ki se v našem vzorcu ne pojavijo, verjetnost malenkost povečamo.

In [ ]:
verjetnosti_korenov_po_zanrih = pojavitve_korenov_po_zanrih / zanri.groupby('zanr').size() + 0.001

Poglejmo, kaj so najpogostejši koreni pri nekaj žanrih:

In [ ]:
verjetnosti_korenov_po_zanrih.Crime.sort_values(ascending=False).head(20)

In [ ]:
verjetnosti_korenov_po_zanrih.Romance.sort_values(ascending=False).head(20)

In [ ]:
verjetnosti_korenov_po_zanrih['Sci-Fi'].sort_values(ascending=False).head(20)

Žanre sedaj določimo tako, da za vsak žanr pomnožimo verjetnost žanra in pogojne verjetnosti vseh korenov, ki nastopajo v opisu filma.

In [ ]:
def doloci_zanre(opis):
    faktorji_zanrov = verjetnosti_zanrov * verjetnosti_korenov_po_zanrih[
        verjetnosti_korenov_po_zanrih.index.isin(
            koreni_besed(opis)
        )
    ].prod()
    faktorji_zanrov /= max(faktorji_zanrov)
    return faktorji_zanrov.sort_values(ascending=False).head(5)

In [ ]:
doloci_zanre('Alien space ship appears above Slovenia.')

In [ ]:
doloci_zanre('A story about a young mathematician, who discovers her artistic side')